# Read the xlsx sheets for without judgements

In [62]:
import os, json
import csv
import pandas as pd

In [63]:
RUNFILES = {
    "auggen": set([
    "arrange-final",
    "cook-desperate",
    "figure-civilian",
    "keep-select",
    "relevant-testimony",
    "store-ethics",
    "viewer-whisper",
    "bite-stir",
    "criteria-african",
    "gently-disagree",
    "neither-fitness",
    "review-basic",
    "that-darkness",
    "both-rate",
    "editor-elsewhere",
    "image-climate",
    "peer-necessarily",
    "sake-frame",
    "transportation-error",
    "closer-submit",
    "entrance-population",
    "internal-withdraw",
    "radical-twice",
    "shock-description",
    "vary-occasion",
]),

"gen": set([
"activity-manufacturer",
"bath-begin",
"degree-mental",
"flag-square",
"listen-promise",
"satellite-gear",
"truth-muslim",
"anger-interpretation",
"battery-founder",
"despite-deer",
"flight-enhance",
"mixture-pause",
"scheme-presidential",
"usual-disaster",
"angle-apartment",
"best-disaster",
"dirty-shop",
"hear-arrangement",
"passenger-snow",
"shortly-seek",
"worry-repeat",
"arab-exciting",
"captain-wage",
"distinction-standard",
"image-hall",
"past-fifth",
"sport-broad",
"area-goodbye",
"child-poison",
"distinguish-fitness",
"institution-painting",
"plant-indeed",
"spread-retirement",
"around-figure",
"commercial-childhood",
"eight-existence",
"jacket-responsibility",
"previously-accept",
"status-produce",
"attach-over",
"concert-matter",
"elect-lamp",
"know-author",
"protest-suspect",
"temperature-thought",
"badly-salary",
"contemporary-dispute",
"existing-engineering",
"laboratory-movie",
"pure-lawyer",
"trend-shoot"
    ])
}

In [64]:
orig_to_anon_map = {}

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/tags.auggen", "r") as f:
    auggen_run_tags = [line.strip() for line in f.readlines()]
    for auggen_run_tag in auggen_run_tags:
        orig_run_id = auggen_run_tag.split(" ")[-1].strip()
        anon_run_id = auggen_run_tag.split(" ")[1].strip()
        orig_to_anon_map[orig_run_id] = anon_run_id

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/tags.gen", "r") as f:
    gen_run_tags = [line.strip() for line in f.readlines()]
    for gen_run_tag in gen_run_tags:
        orig_run_id = gen_run_tag.split(" ")[-1].strip()
        anon_run_id = gen_run_tag.split(" ")[1].strip()
        orig_to_anon_map[orig_run_id] = anon_run_id

print(f"Total anonymized run IDs: {len(orig_to_anon_map)}")
    

Total anonymized run IDs: 76


In [65]:
run_id_to_orig_map = {}

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/auggen-filename-run_id", "r") as f:
    auggen_run_tags = [line.strip() for line in f.readlines()]
    for auggen_run_tag in auggen_run_tags:
        run_id = auggen_run_tag.split(" ")[-1].strip()
        filenmae = auggen_run_tag.split(" ")[0].strip()
        if run_id in run_id_to_orig_map:
            run_id_to_orig_map[run_id].append(filenmae)
        else:
            run_id_to_orig_map[run_id] = [filenmae]

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/gen-filename-run_id", "r") as f:
    gen_run_tags = [line.strip() for line in f.readlines()]
    for gen_run_tag in gen_run_tags:
        run_id = gen_run_tag.split(" ")[-1].strip()
        filenmae = gen_run_tag.split(" ")[0].strip()
        if run_id in run_id_to_orig_map:
            run_id_to_orig_map[run_id].append(filenmae)
        else:
            run_id_to_orig_map[run_id] = [filenmae]

print(f"Total anonymized run IDs: {len(run_id_to_orig_map)}")

Total anonymized run IDs: 67


In [66]:
run_id_to_orig_map["ag-run-1-JH"]

['NITA_AG_JH', 'ag-run-1-JH']

In [ ]:
def get_anonymous_run_id(run_id, orig_to_anon_map, run_id_to_orig_map):
    """Get the anonymous run ID for a given run ID."""

    all_ids = []

    if run_id in RUNFILES["auggen"] or run_id in RUNFILES["gen"]:
        all_ids.append(run_id)
    
    if run_id in orig_to_anon_map:
        anon_run_id = orig_to_anon_map[run_id]
        all_ids.append(anon_run_id)
        
    if run_id in run_id_to_orig_map:
        original_run_id_list = run_id_to_orig_map[run_id]

        for original_run_id in original_run_id_list:
            anon_run_id = orig_to_anon_map.get(original_run_id, None)
            if anon_run_id is not None:
                all_ids.append(anon_run_id)
    
    return list(set(all_ids))

print(get_anonymous_run_id("ag-run-1-JH", orig_to_anon_map, run_id_to_orig_map))

['mixture-pause', 'transportation-error']


In [68]:
def load_runfile(runfile_path):
    runfile = {}
    with open(runfile_path, "r") as f:
        for line in f:
            line = json.loads(line)
            topic_id = str(line["metadata"]["narrative_id"])
            if topic_id not in runfile:
                if "answer" in line:
                    runfile[topic_id] = {"answer": line["answer"], "references": line.get("references", [])}
                elif "responses" in line:
                    runfile[topic_id] = {"answer": line["responses"], "references": line.get("references", [])}
                else:
                    print(f"Topic ID: {topic_id} has no answer or responses")
    return runfile

In [85]:
def figure_out_which_anonymous_run_id(
    anonymous_run_ids: list[str],
    topic_id: str,
    sentence: str,
    sentence_id: int,
) -> str:
    """Figure out which anonymous run ID to use for a given run ID."""
    RUNFILE_DIR = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/"
    
    for anonymous_run_id in anonymous_run_ids:
        # load the results file for the anonymous run ID
        task = "auggen" if anonymous_run_id in RUNFILES["auggen"] else "gen"
        results = load_runfile(os.path.join(RUNFILE_DIR, f"{task}/{anonymous_run_id}"))
        try:
            sentence_anonymous = results[topic_id]["answer"][sentence_id]['text']
            sentence_anonymous = sentence_anonymous.replace('\r', '').replace('\n', '').strip()
            sentence = sentence.replace('\r', '').replace('\n', '').strip()

            if sentence == sentence_anonymous:
                return anonymous_run_id
        except:
            pass
    
    return None

In [96]:
without_prediction_dir = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/rag_assessment/completed"

topicwise_results = {}
# read all excel sheets one by one from the directory
for file in os.listdir(without_prediction_dir):
    complete_filepath = os.path.join(without_prediction_dir, file)
    topic_id = str(file.split("_")[0].strip())
    topicwise_results[topic_id] = {}
    
    if complete_filepath.endswith(".xlsx"):
        # Index(['CITED PASSAGE', 'SENTENCE', 'FULL', 'PARTIAL', 'NONE', 'SENTENCE CONTEXT', 'COMPLETED?', 'TASK', 'RUNID', 'DOCID', 'ANSWERID'], dtype='object')
        print(f"File: {file}")
        
        df = pd.read_excel(complete_filepath)
        for index, row in df.iterrows():
            run_id = row["RUNID"]
            sentence = row["SENTENCE"] if not pd.isna(row["SENTENCE"]) else ""
            sentence_id = row['ANSWERID']
            doc_id = row['DOCID']

            support_score = "-1"
            # check if its under full, partial or none
            # check if any or the rows is not NAN
            if row['FULL'] in ["x", "X"]:
                support_score = '2'
            elif row['PARTIAL'] in ["x", "X"]:
                support_score = '1'
            elif row['NONE'] in ["x", "X"]:
                support_score = '0'
            
            if run_id == "grilllab-agentic-gpt4_1-5-larf-v2":
                run_id = "grilllab-gpt45-gen"

            anonymous_run_ids = get_anonymous_run_id(run_id, orig_to_anon_map, run_id_to_orig_map)
            anonymous_run_id = None

            if len(anonymous_run_ids) == 0:
                print(f"No anonymous run ID found for run ID: {run_id}")
                continue

            elif len(anonymous_run_ids) > 1:
                anonymous_run_id = figure_out_which_anonymous_run_id(anonymous_run_ids, topic_id, sentence, sentence_id)
            
            elif len(anonymous_run_ids) == 1:
                anonymous_run_id = anonymous_run_ids[0]

            if anonymous_run_id is None:
                print(f"No anonymous run ID found for run ID: {run_id}")
                continue

            if anonymous_run_id not in topicwise_results[topic_id]:

                topicwise_results[topic_id][anonymous_run_id] = {
                    "topic_id": topic_id,
                    "run_id": anonymous_run_id,
                    "sentences": [{
                            "sentenceID": sentence_id,
                            "text": sentence,
                            "citations": [{
                                'citationID': 0,
                                'reference': doc_id,
                                'support': support_score,
                            }],
                        }
                    ]
                }
            
            # already the sentence has been defined
            elif anonymous_run_id in topicwise_results[topic_id]: 
                topicwise_results[topic_id][anonymous_run_id]["sentences"].append(
                    {
                        "sentenceID": sentence_id,
                        "text": sentence,
                        "citations": [
                            {
                                'citationID': 0,
                                'reference': doc_id,
                                'support': support_score,
                            }
                        ]
                    }
                )

File: 14_ I_m interested in sports_ societal impact, particularly concerning athlete compensation, inclusion,_.xlsx
File: 72_ I want to understand why deforestation is such a major problem. Specifically, how does it impact the.xlsx
File: 93_ I_m interested in learning how the U.S. Constitution and the Bill of Rights safeguard individual civ.xlsx
File: 144_ I_m seeking a comprehensive understanding of financial institutions, specifically why banks like Sil.xlsx
File: 161_ I want to understand the main arguments surrounding abortion and why people hold such different view.xlsx
File: 200_ I want to deeply understand the Holocaust_ what it was, why and how it transpired, who was responsib.xlsx
File: 213_ I_m looking into the Korean War to learn about its origins, how it ended, and why the US got involve.xlsx
File: 224_ I want to understand why people immigrate or become refugees, the challenges they face, and how laws.xlsx
File: 233_ I_m interested in learning how social media affects ment

In [97]:
ALL_RUN_IDS = list(RUNFILES["auggen"]) + list(RUNFILES["gen"])

all_unique_missing_runs = {}

for topic_id in topicwise_results:
    run_ids = set(topicwise_results[topic_id].keys())
    # print("Missing runs for topic ", topic_id, ": ", len(set(ALL_RUN_IDS) - run_ids), " --- ", set(ALL_RUN_IDS) - run_ids)
    print("Missing runs for topic ", topic_id, ": ", len(set(ALL_RUN_IDS) - run_ids))
    for missing_run in set(ALL_RUN_IDS) - run_ids:
        if missing_run not in all_unique_missing_runs:
            all_unique_missing_runs[missing_run] = 0
        all_unique_missing_runs[missing_run] += 1

Missing runs for topic  14 :  1
Missing runs for topic  72 :  2
Missing runs for topic  93 :  3
Missing runs for topic  144 :  24
Missing runs for topic  161 :  25
Missing runs for topic  200 :  26
Missing runs for topic  213 :  24
Missing runs for topic  224 :  26
Missing runs for topic  233 :  25
Missing runs for topic  273 :  1
Missing runs for topic  300 :  25
Missing runs for topic  407 :  1
Missing runs for topic  477 :  25
Missing runs for topic  499 :  23
Missing runs for topic  515 :  23
Missing runs for topic  707 :  23
Missing runs for topic  897 :  23


In [8]:
# all_unique_missing_runs_sorted = sorted(all_unique_missing_runs, key=all_unique_missing_runs.get, reverse=True)
# anon_to_orig_map = {v: k for k, v in orig_to_anon_map.items()}

# for run in all_unique_missing_runs_sorted:
#     original_run_id = anon_to_orig_map.get(run, "UNKNOWN")
#     print(run, original_run_id, all_unique_missing_runs[run])

In [98]:
# load the empty predictions as empty citations and add them to the topicwise_results
RUNFILE_DIR = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/"

for topic_id in topicwise_results:
    run_ids = topicwise_results[topic_id]
    for run_id in run_ids:
        sentences_with_citations = set([sentence.get("sentenceID") for sentence in run_ids[run_id]["sentences"]])
        task = "auggen" if run_id in RUNFILES["auggen"] else "gen"
        results = load_runfile(os.path.join(RUNFILE_DIR, f"{task}/{run_id}"))
        total_sentences = len(results[topic_id]["answer"])

        if total_sentences > len(sentences_with_citations):
            sentence_ids_missing = [idx for idx in range(0, total_sentences) if idx not in sentences_with_citations]
            print(f"topic ID: {topic_id}\tRun ID: {run_id}\tempty citation sentences: {sentence_ids_missing}")

            for idx in sentence_ids_missing:
                topicwise_results[topic_id][run_id]["sentences"].append(
                    {
                        "sentenceID": idx,
                        "text": results[topic_id]["answer"][idx]['text'],
                        "citations": []
                    }
                )

topic ID: 14	Run ID: usual-disaster	empty citation sentences: [0]
topic ID: 14	Run ID: internal-withdraw	empty citation sentences: [1, 5]
topic ID: 14	Run ID: vary-occasion	empty citation sentences: [6]
topic ID: 14	Run ID: peer-necessarily	empty citation sentences: [0, 4, 6, 7]
topic ID: 14	Run ID: around-figure	empty citation sentences: [0]
topic ID: 14	Run ID: review-basic	empty citation sentences: [0]
topic ID: 14	Run ID: existing-engineering	empty citation sentences: [0, 10]
topic ID: 14	Run ID: plant-indeed	empty citation sentences: [0, 5]
topic ID: 14	Run ID: eight-existence	empty citation sentences: [0]
topic ID: 14	Run ID: despite-deer	empty citation sentences: [6]
topic ID: 14	Run ID: concert-matter	empty citation sentences: [13]
topic ID: 14	Run ID: flag-square	empty citation sentences: [0, 3, 4, 6]
topic ID: 14	Run ID: flight-enhance	empty citation sentences: [0, 4]
topic ID: 14	Run ID: gently-disagree	empty citation sentences: [0, 3, 4, 6]
topic ID: 14	Run ID: arrange-fina

In [99]:
# save these judgments to the output directory
with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/results/support_assignment/human_judgments.jsonl", "w") as f:
    for topic_id, results in topicwise_results.items():
        for run_id, value in results.items():
            x = {
                "narrative_id": topic_id,
                "run_id": run_id,
                "sentences": sorted(value["sentences"], key=lambda x: x["sentenceID"])
            }
            f.write(json.dumps(x) + "\n")

In [100]:
## Since we are only evaluating the top 3 runs from each team; lets find those runs

ALL_RUN_IDS = set()
anon_to_orig_map = {v: k for k, v in orig_to_anon_map.items()}

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/auggen-runs.top3.ordered", "r") as f:
    auggen_run_tags = [line.strip() for line in f.readlines()]
    for original_run_id in auggen_run_tags:
        anon_run_id = orig_to_anon_map.get(original_run_id, None)
        if anon_run_id is not None:
            ALL_RUN_IDS.add(anon_run_id)

with open("/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/runs/anon/tags/gen-runs.top3.ordered", "r") as f:
    gen_run_tags = [line.strip() for line in f.readlines()]
    for original_run_id in gen_run_tags:
        anon_run_id = orig_to_anon_map.get(original_run_id, None)
        if anon_run_id is not None:
            ALL_RUN_IDS.add(anon_run_id)

print(f"Total run IDs: {len(ALL_RUN_IDS)}")

Total run IDs: 57


In [101]:
import os

weighted_scores = {-1: 0, 0: 0, 1: 0.5, 2: 1}
hard_scores = {-1: 0, 0: 0, 1: 0, 2: 1}

# Create a directory to store the results
OUTPUT_DIR = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/results/support_assignment/leaderboard/participant_results/human/"
auggen_output_dir = os.path.join(OUTPUT_DIR, "auggen")
gen_output_dir = os.path.join(OUTPUT_DIR, "gen")

# Create the output directory if it does not exist
os.makedirs(auggen_output_dir, exist_ok=True)
os.makedirs(gen_output_dir, exist_ok=True)

# intiliaze the scores for each run ID
all_run_scores = {run_id: {} for run_id in ALL_RUN_IDS}

for topic_id in topicwise_results:
    individual_results = topicwise_results.get(topic_id, {})
    print(f"Processing topic ID: {topic_id} with {len(individual_results)} runs")

    # Iterate over each run ID, sentences
    for run_id, value in individual_results.items():
        
        if run_id not in ALL_RUN_IDS:
            continue
        
        sentences = value.get("sentences", [])
        sent_with_citations, weighted_score, hard_score = 0, 0.0, 0.0
        total_count_sentences = len(sentences)
        
        for sentence in sentences:
            if len(sentence.get('citations')) > 0:
                first_citation = sentence.get('citations')[0]
                support_score = int(first_citation.get('support'))
                
                if support_score > -1:
                    # numerator
                    weighted_score += weighted_scores.get(support_score)
                    hard_score += hard_scores.get(support_score)
                    sent_with_citations += 1
                
                elif support_score == -1:
                    total_count_sentences -= 1
        
         # Write the results to a file
        output_dir = auggen_output_dir if run_id in RUNFILES["auggen"] else gen_output_dir
        original_run_id = anon_to_orig_map.get(run_id)
        with open(f"{output_dir}/{original_run_id}.jsonl", "a") as f:
            values = {
                "topic_id": topic_id,
                "weighted_precision": weighted_score / sent_with_citations if sent_with_citations > 0 else 0,
                "hard_precision": hard_score / sent_with_citations if sent_with_citations > 0 else 0,
                "weighted_recall": weighted_score / total_count_sentences if total_count_sentences > 0 else 0,
                "hard_recall": hard_score / total_count_sentences if total_count_sentences > 0 else 0,
                "sentences": len(sentences)
            }
            f.write(json.dumps(values) + "\n")
    
    ## add the missing run IDs for the topic ID
    for run_id_temp in ALL_RUN_IDS:
        if run_id_temp not in individual_results:
            print(f"Run ID: {run_id_temp} not there in topic ID: {topic_id}")
            output_dir = auggen_output_dir if run_id_temp in RUNFILES["auggen"] else gen_output_dir
            original_run_id = anon_to_orig_map.get(run_id_temp)
            with open(f"{output_dir}/{original_run_id}.jsonl", "a") as f:
                values = {
                    "topic_id": topic_id,
                    "weighted_precision": 0.0,
                    "hard_precision": 0.0,
                    "weighted_recall": 0.0,
                    "hard_recall": 0.0,
                    "sentences": 0.0
                }
                f.write(json.dumps(values) + "\n")

Processing topic ID: 14 with 75 runs
Run ID: angle-apartment not there in topic ID: 14
Processing topic ID: 72 with 74 runs
Run ID: angle-apartment not there in topic ID: 72
Run ID: attach-over not there in topic ID: 72
Processing topic ID: 93 with 73 runs
Run ID: angle-apartment not there in topic ID: 93
Run ID: degree-mental not there in topic ID: 93
Processing topic ID: 144 with 52 runs
Run ID: bath-begin not there in topic ID: 144
Run ID: flight-enhance not there in topic ID: 144
Run ID: gently-disagree not there in topic ID: 144
Run ID: transportation-error not there in topic ID: 144
Run ID: worry-repeat not there in topic ID: 144
Run ID: activity-manufacturer not there in topic ID: 144
Processing topic ID: 161 with 51 runs
Run ID: hear-arrangement not there in topic ID: 161
Run ID: bath-begin not there in topic ID: 161
Run ID: flight-enhance not there in topic ID: 161
Run ID: gently-disagree not there in topic ID: 161
Run ID: transportation-error not there in topic ID: 161
Run ID

### COMPUTE AVERAGES

In [102]:
# compute the average scores for each run ID in the merged file directory
import glob

total_topics = 17

OUTPUT_DIRS = {
    "auggen": auggen_output_dir,
    "gen": gen_output_dir
}

for run_id in ALL_RUN_IDS:
    merged_results = []
    task = "auggen" if run_id in RUNFILES["auggen"] else "gen"
    output_dir = OUTPUT_DIRS[task]
    original_run_id = anon_to_orig_map.get(run_id)
    for file in glob.glob(f"{output_dir}/{original_run_id}.jsonl"):
        with open(file, "r") as f:
            for line in f:
                merged_results.append(json.loads(line))
    
    with open(f"{output_dir}/{original_run_id}.jsonl", "a") as f:
        values = {
            "topic_id": "all",
            "weighted_precision": sum([v["weighted_precision"] for v in merged_results]) / total_topics,
            "hard_precision": sum([v["hard_precision"] for v in merged_results]) / total_topics,
            "weighted_recall": sum([v["weighted_recall"] for v in merged_results]) / total_topics,
            "hard_recall": sum([v["hard_recall"] for v in merged_results]) / total_topics,
            "sentences": sum([v["sentences"] for v in merged_results]) / total_topics,
            "topics_evaluated": total_topics - sum(1 for v in merged_results if v["sentences"] == 0),
        }
        f.write(json.dumps(values) + "\n")

In [94]:
# print the final scores for each run ID as a csv file
import pandas as pd

for task in ["auggen", "gen"]:
    output_dir = OUTPUT_DIRS[task]
    all_results = []
    for file in glob.glob(f"{output_dir}/*.jsonl"):
        run_id = os.path.basename(file).replace(".jsonl", "")
        print(f"Processing run ID: {run_id}")
        with open(file, "r") as f:
            for line in f:
                json_line = json.loads(line)

                if json_line["topic_id"] == "all":
                    all_results.append({
                        "run_id": run_id, 
                        "Weighted Precision": round(json_line["weighted_precision"], 3),
                        "Hard Precision": round(json_line["hard_precision"], 3),
                        "Weighted Recall": round(json_line["weighted_recall"], 3),
                        "Hard Recall": round(json_line["hard_recall"], 3),
                        "Sentences": round(json_line["sentences"], 3),
                        "Topics Evaluated": json_line["topics_evaluated"],
                    })
    
    df = pd.DataFrame(all_results)
    df = df.sort_values(by=["Weighted Precision"], ascending=False)
    df.to_csv(f"{output_dir}/../{task}_support_results_17_topics.csv", index=False)

Processing run ID: ag-v2-gpt
Processing run ID: wingii-3-rl-refined
Processing run ID: ag-v1-gpt
Processing run ID: no-llm-refined
Processing run ID: ag-v2-llama
Processing run ID: wingii-v3-gpt
Processing run ID: gpt41
Processing run ID: IDACCS-nugg-gpt-4-1
Processing run ID: Rerank-Top50_v3
Processing run ID: IDACCSabstrct-gpt4-1
Processing run ID: IDACCS-hybrid-gpt4-1
Processing run ID: Rerank-Top50_v2
Processing run ID: cluster-generation
Processing run ID: nugget-generation
Processing run ID: cru-ablR
Processing run ID: cru-ansR-conf
Processing run ID: rag25_qwen3_50_ag
Processing run ID: rag25_qwen3_20_ag
Processing run ID: cru-ansR
Processing run ID: NITA_AG_JH
Processing run ID: uema2lab_B4
Processing run ID: hltcoe-gpt5.searcher
Processing run ID: r_4method_ag_gpt41
Processing run ID: hltcoe-lg.qwen
Processing run ID: KG-AG-1
Processing run ID: hltcoe-lg.searcher
Processing run ID: lg_nt_q4d12l3_c
Processing run ID: bm25-rz7b-2025a
Processing run ID: no-reranker
Processing run

## Merge per-topic runs across participants

In [33]:
topic_ids = [
    "14","72","93","144","161","200","213","224","233","273","300","407","477","499","515","707","897","all"
]

In [34]:
### 
import os
import numpy as np

OUTPUT_DIR = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/results/support_assignment/leaderboard/participant_results/human/"

# Create a directory to store the results
topic_output_dir = os.path.join(OUTPUT_DIR, "per-topic")

OUTPUT_DIRS = {
    "auggen": topic_output_dir,
    "gen": topic_output_dir
}

# Create the output directory if it does not exist
os.makedirs(topic_output_dir, exist_ok=True)

# merge the jsonl files for each directory
import glob, os, json

merged_results = {topic_id: {} for topic_id in topic_ids}

for run_id in ALL_RUN_IDS:
    task = "auggen" if run_id in RUNFILES["auggen"] else "gen"
    original_run_id = anon_to_orig_map.get(run_id)
    for file in glob.glob(f"{OUTPUT_DIR}/{task}/{original_run_id}.jsonl"): 
        with open(file, "r") as f:
            for line in f:
                topic_id = json.loads(line)["topic_id"]
                merged_results[topic_id][original_run_id] = json.loads(line)

print(f"Total topics in merged results: {len(merged_results)}")

# write the results to the output directory
with open(f"{topic_output_dir}/mean_topicwise_scores.jsonl", "w") as f:
    for topic_id in topic_ids:
        # runfiles for each topic
        runfiles = merged_results[topic_id]
        ## compute average scores across all run_ids
        total_runfiles = len(runfiles)
        
        values = {
            "topic_id": topic_id,
            "weighted_precision": sum([v["weighted_precision"] for v in runfiles.values()]) / total_runfiles,
            "hard_precision": sum([v["hard_precision"] for v in runfiles.values()]) / total_runfiles,
            "weighted_recall": sum([v["weighted_recall"] for v in runfiles.values()]) / total_runfiles,
            "hard_recall": sum([v["hard_recall"] for v in runfiles.values()]) / total_runfiles,
            "sentences": sum([v["sentences"] for v in runfiles.values()]) / total_runfiles
        }
        
        f.write(json.dumps(values) + "\n")

# write the results to the output directory
with open(f"{topic_output_dir}/median_topicwise_scores.jsonl", "w") as f:
    for topic_id in topic_ids:
        # runfiles for each topic
        runfiles = merged_results[topic_id]
        ## compute average scores across all run_ids
        total_runfiles = len(runfiles)
        
        # get the scores for each run_id
        hard_precision = np.array([v["hard_precision"] for v in runfiles.values()])
        weighted_precision = np.array([v["weighted_precision"] for v in runfiles.values()])
        weighted_recall = np.array([v["weighted_recall"] for v in runfiles.values()])
        hard_recall = np.array([v["hard_recall"] for v in runfiles.values()])
        sentences = np.array([v["sentences"] for v in runfiles.values()])

        values = {
            "topic_id": topic_id,
            "weighted_precision": float(np.median(weighted_precision)),
            "hard_precision": float(np.median(hard_precision)),
            "weighted_recall": float(np.median(weighted_recall)),
            "hard_recall": float(np.median(hard_recall)),
            "sentences": float(np.median(sentences))
        }
        
        f.write(json.dumps(values) + "\n")

# write the results to the output directory
with open(f"{topic_output_dir}/max_topicwise_scores.jsonl", "w") as f:
    for topic_id in topic_ids:
        # runfiles for each topic
        runfiles = merged_results[topic_id]
        ## compute average scores across all run_ids
        total_runfiles = len(runfiles)
        
        # get the scores for each run_id
        hard_precision = np.array([v["hard_precision"] for v in runfiles.values()])
        weighted_precision = np.array([v["weighted_precision"] for v in runfiles.values()])
        weighted_recall = np.array([v["weighted_recall"] for v in runfiles.values()])
        hard_recall = np.array([v["hard_recall"] for v in runfiles.values()])
        sentences = np.array([v["sentences"] for v in runfiles.values()])

        values = {
            "topic_id": topic_id,
            "weighted_precision": float(np.max(weighted_precision)),
            "hard_precision": float(np.max(hard_precision)),
            "weighted_recall": float(np.max(weighted_recall)),
            "hard_recall": float(np.max(hard_recall)),
            "sentences": float(np.max(sentences))
        }
        
        f.write(json.dumps(values) + "\n")

# write the results to the output directory
with open(f"{topic_output_dir}/min_topicwise_scores.jsonl", "w") as f:
    for topic_id in topic_ids:
        # runfiles for each topic
        runfiles = merged_results[topic_id]
        ## compute average scores across all run_ids
        total_runfiles = len(runfiles)
        
        # get the scores for each run_id
        hard_precision = np.array([v["hard_precision"] for v in runfiles.values()])
        weighted_precision = np.array([v["weighted_precision"] for v in runfiles.values()])
        weighted_recall = np.array([v["weighted_recall"] for v in runfiles.values()])
        hard_recall = np.array([v["hard_recall"] for v in runfiles.values()])
        sentences = np.array([v["sentences"] for v in runfiles.values()])

        values = {
            "topic_id": topic_id,
            "weighted_precision": float(np.min(weighted_precision)),
            "hard_precision": float(np.min(hard_precision)),
            "weighted_recall": float(np.min(weighted_recall)),
            "hard_recall": float(np.min(hard_recall)),
            "sentences": float(np.min(sentences))
        }
        
        f.write(json.dumps(values) + "\n")
    

Total topics in merged results: 18


In [35]:
OUTPUT_DIRS = {
    "auggen": auggen_output_dir,
    "gen": gen_output_dir
}

OUTPUT_FINAL_DIR = "/store/scratch/n3thakur/trec-rag-2024/trec2024-rag/support_eval/2025/trec25-rag/results/final_dump/"

for task in ["auggen", "gen"]:
    with open(f"{OUTPUT_FINAL_DIR}/{task}_support_nist_w_topics.txt", "w") as fout:
        output_dir = OUTPUT_DIRS[task]
        all_results = []
        anonymous_run_ids = [orig_to_anon_map.get(os.path.basename(file).replace(".jsonl", "")) for file in glob.glob(f"{output_dir}/*.jsonl")]

        for run_id in sorted(anonymous_run_ids):
            original_run_id = anon_to_orig_map.get(run_id)
            for metric in ["weighted_precision", "weighted_recall"]:
                with open(f"{output_dir}/{original_run_id}.jsonl", "r") as f:
                    for line in f:
                        json_line = json.loads(line)
                        if json_line["topic_id"] != "all":
                            topic_id = json_line["topic_id"]
                            if round(json_line[metric], 3) > 0.00001:
                                fout.write(f"{run_id} {topic_id} {metric} {round(json_line[metric], 3)}")
                                fout.write("\n")